In [2]:
from bibleabbr import *

In [3]:
import numpy as np
import shorttext
import sqlite3

# Scripture Loaded

In [3]:
dbconn = sqlite3.connect('ESVCrossway.sqlite')

In [4]:
from functools import reduce
from nltk import sent_tokenize

def retrieve_docs_as_biblechapters(dbconn):
    cursor = dbconn.cursor()
    for book in wholebible_book_iterator():
        for chap in range(1, numchaps[book]+1):
            doc_label = book+'_'+str(chap)
            result = cursor.execute('select scripture from bible where book is "'+book+'" and chapter='+str(chap))
            doc_text = reduce(lambda s1, s2: ' '.join([s1, s2]), [texttuple[0] for texttuple in result])
            yield doc_label, doc_text
    cursor.close()
    

In [5]:
from collections import defaultdict

classdict = defaultdict(lambda : [])
for bible_chap, verses in retrieve_docs_as_biblechapters(dbconn):
    for verse in sent_tokenize(verses):
        classdict[bible_chap] += [verse]

# C-LSTM

In [10]:
wmodel = shorttext.utils.load_word2vec_model('wordembedD/GoogleNews-vectors-negative300.bin')

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [12]:
kmodel = shorttext.classifiers.CLSTMWordEmbed(len(classdict.keys()), vecsize=wmodel.vector_size)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wmodel)

In [14]:
classifier.train(classdict, kmodel)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
35885/35885 [==============================] - 170s 5ms/step - loss: 6.1304
Epoch 2/10
35885/35885 [==============================] - 160s 4ms/step - loss: 4.4199
Epoch 3/10
35885/35885 [==============================] - 151s 4ms/step - loss: 3.0092
Epoch 4/10
35885/35885 [==============================] - 151s 4ms/step - loss: 1.7209
Epoch 5/10
35885/35885 [==============================] - 150s 4ms/step - loss: 0.8826
Epoch 6/10
35885/35885 [==============================] - 150s 4ms/step - loss: 0.5396
Epoch 7/10
35885/35885 [==============================] - 151s 4ms/step - loss: 0.4121
Epoch 8/10
35885/35885 [==============================] - 149s 4ms/step - loss: 0.3777
Epoch 9/10
35885/35885 [==============================] - 150s 4ms/step - loss: 0.3461
Epoch 10/10
35885/35885 [==============================] - 149s 4ms/step - loss: 0.3249


In [15]:
classifier.save_compact_model('biblechap_CLSTM.pkl')

# Prediction

In [4]:
import shorttext

wmodel = shorttext.utils.load_word2vec_model('wordembedD/GoogleNews-vectors-negative300.bin')

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
classifier = shorttext.smartload_compact_model('biblechap_CLSTM.pkl', wmodel)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
sorted(classifier.score('justice').items(), key=lambda item: item[1], reverse=True)[:10]

[('mic_3', 0.0896541),
 ('is_30', 0.031264406),
 ('eze_45', 0.028011644),
 ('is_56', 0.022064975),
 ('2ch_19', 0.02097103),
 ('lk_18', 0.019383995),
 ('am_5', 0.01881196),
 ('mt_12', 0.015013556),
 ('ec_3', 0.014086917),
 ('1sa_15', 0.013358957)]